In [1]:
# autoreload

%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../../src")

import SimpleITK as sitk
from glob import glob
from pathlib import Path
import json
import pandas as pd
import os
import tqdm
import numpy as np
import pickle
from skimage import measure
from metrics import get_aneurysm_diameter, get_gt_case_props, compute_metrics_glia, print_metrics, process_label_files_centr, compute_metrics_centroid_det_based

np.set_printoptions(precision=4, suppress=True)

## GLIA-Net evaluation


In [2]:
paths_gts = {
    "internal": "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/internal_0.4_crop.csv",
    "external": "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/external_0.4_crop.csv",
    "hospital": "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/gt/hospital_crop_0.4.csv",
}

paths_glia = {
    "internal": "/home/ceballosarroyo.a/workspace/medical/.outputs_to_compare/glia_internal/",
    "external": "/home/ceballosarroyo.a/workspace/medical/.outputs_to_compare/glia_external/",
    "hospital": "/home/ceballosarroyo.a/workspace/medical/.outputs_to_compare/glia_hospital/",
}

paths_labels = {
    "internal": "/work/vig/Datasets/aneurysm/internal_test/crop_0.4_label",
    "external": "/work/vig/Datasets/aneurysm/external/crop_0.4_label",
    "hospital": "/work/vig/Datasets/aneurysm/hospital/crop_0.4_label",
}

In [3]:
mode = "hospital"
path_props_cache = Path(f"../../.cache/props_{mode}_gt.pickle")
path_props_label_cache = Path(f"../../.cache/props_{mode}_label.pickle")
path_glia = Path(
    f"/home/ceballosarroyo.a/workspace/medical/.outputs_to_compare/glia_{mode}/"
)

files_glia = list(path_glia.glob("*.nii.gz"))
files_label = list(Path(paths_labels[mode]).glob("*.nii.gz"))

df_internal = pd.read_csv(paths_gts[mode])
df_internal.head()

,seriesuid,coordX,coordY,coordZ,w,h,d,lesion,volume,min_axis,maj_axis
0,CA_00000_0000.nii.gz,305.0,319.5,169.5,17,18,14,aneurysm,1630.0,12.982818,20.656223
1,CA_00000_0000.nii.gz,315.0,317.0,179.0,5,5,1,aneurysm,25.0,0.000000,6.324555
2,CA_00001_0000.nii.gz,284.5,334.5,145.5,14,16,12,aneurysm,1360.0,13.303996,16.579593
3,CA_00002_0000.nii.gz,332.5,339.5,141.0,6,6,3,aneurysm,102.0,3.554910,7.637626
4,CA_00002_0000.nii.gz,305.5,354.5,161.5,6,6,2,aneurysm,72.0,2.236068,7.637626


In [4]:
pred_props, label_props = process_label_files_centr(
    files_glia, files_label, path_props_cache, path_props_label_cache
)
# retrieve volumes for both preds and gts
volumes_pred = []
volumes_label = []
for key in pred_props.keys():
    for det in pred_props[key]:
        volumes_pred.append(det["area"])
    for det in label_props[key]:
        volumes_label.append(det["area"])

df_results, aneurysms_found = compute_metrics_glia(
    files_glia, df_internal, pred_props, metadata=None
)
df_internal["volume"] = volumes_label
df_internal["detected"] = aneurysms_found

In [5]:
print(f"Mode {mode.upper()}\n")
print_metrics(df_results)

Mode HOSPITAL

Total cases:  38
Healthy cases:  0
Sick cases:  38
FP Rate: 5.421052631578948
Recall: 0.4482758620689655
FP Rate (Healthy patients): nan
Patient-level sensitivity: 0 / 0


/work/vig/alberto/medical/cta-det2/notebooks/stats/../../src/metrics.py:204: RuntimeWarning: invalid value encountered in scalar divide
  df_results_healthy["fp"].sum() / len(df_results_healthy),


In [6]:
# each voxel is 0.4mm spacing, to transform to voxel coords we..

# 1. divide by 0.4
# 2. round to the nearest integer

## Detection-based models evaluation


In [7]:
paths_gts = {
    "internal": "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/internal_0.4_crop.csv",
    "external": "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/external_0.4_crop.csv",
    "hospital": "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/gt/hospital_crop_0.4.csv",
}

paths_labels = {
    "internal": "/work/vig/Datasets/aneurysm/internal_test/crop_0.4_label",
    "external": "/work/vig/Datasets/aneurysm/external/crop_0.4_label",
    "hospital": "/work/vig/Datasets/aneurysm/hospital/crop_0.4_label",
}

In [15]:
exp = "adeform_decoder_only_non_rec_crop_vessel_pe_gpe"  # final
exp = "adeform_decoder_only_non_rec_crop_vessel_pe_gpe_v2"  # 56
exp = "deform_decoder_only_non_rec_BEST_cropinf"  # 30

In [39]:
exp = exp_base = "deform_decoder_only_non_rec_BEST_cropinf"
cp = "30k"
modes = ["internal", "external", "hospital"]
t = 0.95
for mode in modes:
    if mode == "external":
        exp = exp_base + "_EXT"
    elif mode == "hospital":
        exp = exp_base + "_PRIV"

    path_pred = f"/home/ceballosarroyo.a/workspace/medical/cta-det2/outputs/{exp}/inference_{cp}/predict.csv"
    # check if it exists
    if not os.path.exists(path_pred):
        print(f"File {path_pred} does not exist for mode {mode}")
        continue
    files_label = list(Path(paths_labels[mode]).glob("*.nii.gz"))
    df_gt = pd.read_csv(paths_gts[mode])
    df_pred = pd.read_csv(path_pred)
    df_pred = df_pred[df_pred["probability"] > t]
    df_results, aneurysms_found = compute_metrics_centroid_det_based(
        files_label, df_gt, df_pred, radius_factor=3
    )
    print(f"Mode: {mode.upper()}\n")
    print_metrics(df_results)
    print("\n" + "_" * 20)

Mode: INTERNAL

Total cases:  152
Healthy cases:  50
Sick cases:  102
FP Rate: 0.5986842105263158
Recall: 0.9841269841269841
FP Rate (Healthy patients): 0.68
Patient-level sensitivity: 30 / 50

____________________
Mode: EXTERNAL

Total cases:  138
Healthy cases:  46
Sick cases:  92
FP Rate: 0.9347826086956522
Recall: 0.9801980198019802
FP Rate (Healthy patients): 0.5434782608695652
Patient-level sensitivity: 29 / 46

____________________
Mode: HOSPITAL

Total cases:  38
Healthy cases:  0
Sick cases:  38
FP Rate: 2.3947368421052633
Recall: 0.7758620689655172
FP Rate (Healthy patients): nan
Patient-level sensitivity: 0 / 0

____________________


/work/vig/alberto/medical/cta-det2/notebooks/stats/../../src/metrics.py:204: RuntimeWarning: invalid value encountered in scalar divide
  df_results_healthy["fp"].sum() / len(df_results_healthy),


Mode EXTERNAL

Total cases:  138
Healthy cases:  46
Sick cases:  92
FP Rate: 1.3840579710144927
Recall: 0.9603960396039604
FP Rate (Healthy patients): 0.6304347826086957
Patient-level sensitivity: 26 / 46


In [95]:
df_results_internal = pd.DataFrame(rows_ours, columns=cols)
df_results_healthy = df_results_internal[
    (df_results_internal["tp"] == 0) & (df_results_internal["fn"] == 0)
]
len(df_results_healthy[df_results_healthy["fp"] == 0]), len(df_results_healthy),

(28, 50)

In [96]:
df_results_internal

,tp,fp,fn
0,0,0,0
1,0,0,0
2,0,0,0
3,0,1,0
4,0,0,0
...,...,...,...
147,1,3,0
148,1,0,0
149,0,1,0
150,2,0,0


In [97]:
df_results_internal["total_aneurysms"] = (
    df_results_internal["tp"] + df_results_internal["fn"]
)
len(
    df_results_internal[
        (df_results_internal["tp"] > 0) & (df_results_internal["fn"] == 0)
    ]
), len(df_results_internal[df_results_internal["total_aneurysms"] > 0]),

(97, 102)

In [98]:
df_results_healthy["fp"].sum() / len(df_results_healthy)

0.68

In [99]:
aggregte = df_results_internal.sum()
aggregte["tp"] / len(df_gt), len(df_gt), aggregte["fp"] / len(files_glia), len(
    files_glia
)

(0.9603174603174603, 126, 2.9210526315789473, 38)

In [100]:
true_positive_count = np.sum(matches_scan, axis=1)
true_positive_count

array([1])

In [101]:
649 / 126

5.150793650793651

In [102]:
sum(aneurysms_found)

130

In [103]:
np.unique(label)

NameError: name 'label' is not defined

In [ ]:
cols = ["tp", "fp", "fn"]
rows_ours = []
aneurysms_found = []
volume_in_voxels = []
for i in range(0, len(files_glia)):

    case_name = files_glia_internal[i].name
    df_case = df_gt[df_gt["seriesuid"] == case_name]
    df_case_pred = df_ours[df_ours["seriesuid"] == case_name]

    # get all connected components

    if len(df_case) == 0:
        new_row = [0, len(df_case_pred), 0]
        rows_ours.append([0, len(df_case_pred), 0])

    else:
        aneurysms = df_case.values
        preds = df_case_pred.values
        matches_scan = []
        aneurysms = df_case.values
        tps = 0
        fps = 0
        fns = len(aneurysms)
        # seriesuid,coordX,coordY,coordZ,w,h,d,lesion
        for aneurysm in aneurysms:
            gt_z, gt_y, gt_x, gt_w, gt_h, gt_d = aneurysm[1:-1]

            center_gt = [gt_x, gt_y, gt_z]

            radius_gt = max(gt_w / 2, gt_h / 2, gt_d / 2)
            found = False
            matches_aneurysm = []
            for prop in preds:
                pred_x, pred_y, pred_z, pred_w, pred_h, pred_d = prop[2:]
                center_pred = [pred_x, pred_y, pred_z]
                distance_gt = (
                    np.linalg.norm(np.array(center_gt) - np.array(center_pred)) / 0.4
                )
                radius = np.max((pred_w / 2, pred_h / 2, pred_d / 2))
                print(distance_gt, radius_gt, radius)
                if distance_gt < radius_gt + radius:
                    matches_aneurysm.append(True)
                else:
                    matches_aneurysm.append(False)
            matches_scan.append(matches_aneurysm)
        matches_scan = np.array(matches_scan).astype(int)
        true_positive_count = np.sum(matches_scan, axis=1)
        detected_aneurysms_scan = list(true_positive_count)
        aneurysms_found += detected_aneurysms_scan
        true_positive_count = np.sum(true_positive_count > 0)
        false_positive_count = np.sum(matches_scan, axis=0)
        false_positive_count = np.sum(false_positive_count == 0)
        false_negative_count = len(aneurysms) - true_positive_count
        new_row = [true_positive_count, false_positive_count, false_negative_count]

        rows_ours.append(
            [true_positive_count, false_positive_count, false_negative_count]
        )
    print("\n", cols)
    print(new_row)

In [ ]:
path_glia_external = Path(
    "/home/ceballosarroyo.a/workspace/medical/.outputs_to_compare/glia_external/"
)
files_glia = list(path_glia_external.glob("*.nii.gz"))

external_csv = (
    "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/external_0.4_crop.csv"
)
df_gt = pd.read_csv(external_csv)
pred_props = {}

for i in range(0, len(files_glia)):
    case_name = files_glia[i].name
    print(case_name)
    pred_props[case_name] = {}
    header = sitk.ReadImage(str(files_glia[i]))
    label = sitk.GetArrayFromImage(header)
    label = label.astype(np.uint8)
    all_labels = measure.label(label, background=0)
    props = measure.regionprops(all_labels)
    pred_props[case_name]["props"] = props

In [ ]:
len(files_glia)

138

In [ ]:
import json

cols = ["tp", "fp", "fn"]
rows = []
aneurysms_found = []
volume_in_voxels = []
metadata_file = "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/metadata/external_crop_meta.json"
metadata = json.load(open(metadata_file))
for i in range(0, len(files_glia)):

    case_name = files_glia[i].name
    df_case = df_gt[df_gt["seriesuid"] == case_name]
    spacing = np.array(metadata[case_name]["spacing"])
    origin = np.array(metadata[case_name]["origin"])

    # get all connected components
    props = pred_props[case_name]["props"]

    if len(df_case) == 0:
        new_row = [0, len(props), 0]
        rows.append([0, len(props), 0])

    else:
        matches_scan = []
        aneurysms = df_case.values
        tps = 0
        fps = 0
        fns = len(aneurysms)
        # seriesuid,coordX,coordY,coordZ,w,h,d,lesion
        for aneurysm in aneurysms:
            gt_x, gt_y, gt_z, gt_w, gt_h, gt_d = aneurysm[1:-1]
            center_gt = [gt_x, gt_y, gt_z]
            radius_gt = max(gt_w / 2, gt_h / 2, gt_d / 2)
            found = False
            matches_aneurysm = []
            for prop in props:
                min_z, min_y, min_x, max_z, max_y, max_x = (
                    np.array(prop.bbox) * spacing[0]
                )
                z_pred, y_pred, x_pred = prop.centroid
                volume_in_voxels.append(prop.area)
                center_pred = np.array([x_pred, y_pred, z_pred]) * spacing + origin
                distance_gt = np.linalg.norm(
                    np.array(center_gt) - np.array(center_pred)
                )
                radius = max((max_z - min_z, max_y - min_y, max_x - min_x)) / 2
                print(distance_gt, radius_gt, radius)
                if distance_gt < radius_gt + radius:
                    matches_aneurysm.append(True)
                else:
                    matches_aneurysm.append(False)
            matches_scan.append(matches_aneurysm)
        matches_scan = np.array(matches_scan).astype(int)
        true_positive_count = np.sum(matches_scan, axis=1)
        detected_aneurysms_scan = list(true_positive_count)
        aneurysms_found += detected_aneurysms_scan
        true_positive_count = np.sum(true_positive_count > 0)
        false_positive_count = np.sum(matches_scan, axis=0)
        false_positive_count = np.sum(false_positive_count == 0)
        false_negative_count = len(aneurysms) - true_positive_count
        new_row = [true_positive_count, false_positive_count, false_negative_count]

        rows.append([true_positive_count, false_positive_count, false_negative_count])
    print("\n", cols)
    print(new_row)

30.651083453735197 3.8000000566244125 0.6000000089406967
31.20000046491623 3.8000000566244125 0.4000000059604645
44.04305559765908 3.8000000566244125 3.0000000447034836
0.6583296515800078 3.8000000566244125 3.6000000536441803

 ['tp', 'fp', 'fn']
[1, 3, 0]
12.281539704492285 4.400000065565109 1.0000000149011612

 ['tp', 'fp', 'fn']
[0, 1, 1]
22.04719189571027 4.000000059604645 1.0000000149011612
22.842243365325086 4.000000059604645 1.2000000178813934
2.9484282975417693 4.000000059604645 1.2000000178813934
3.617679324625925 4.000000059604645 1.0000000149011612
2.6030751433526707 4.000000059604645 0.800000011920929
23.93430082300334 4.000000059604645 1.2000000178813934
4.466581287057203 4.000000059604645 1.4000000208616257

 ['tp', 'fp', 'fn']
[1, 3, 0]
61.38594267791256 5.40000008046627 1.0000000149011612
61.05930296406192 5.40000008046627 0.800000011920929
61.49113848911361 5.40000008046627 0.4000000059604645
33.54241591672256 5.40000008046627 1.4000000208616257
32.37875146085598 5.400

In [ ]:
df_results_internal = pd.DataFrame(rows, columns=cols)
df_results_healthy = df_results_internal[
    (df_results_internal["tp"] == 0) & (df_results_internal["fn"] == 0)
]
len(df_results_healthy[df_results_healthy["fp"] == 0]), len(df_results_healthy),

(1, 46)

In [ ]:
df_results_internal["total_aneurysms"] = (
    df_results_internal["tp"] + df_results_internal["fn"]
)
len(
    df_results_internal[
        (df_results_internal["tp"] > 0) & (df_results_internal["fn"] == 0)
    ]
), len(df_results_internal[df_results_internal["total_aneurysms"] > 0]),

(68, 92)

In [ ]:
aggregte = df_results_internal.sum()
aggregte["tp"] / len(df_gt), len(df_gt), aggregte["fp"] / len(files_glia), len(
    files_glia
)

(0.7227722772277227, 101, 5.22463768115942, 138)